In [1]:
%matplotlib inline
%timeit

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import time
import pickle
import datetime
import re
import pyActigraphy

import sys
sys.path.append('/home/ngrav/project/')
from wearables.scripts import utils as wearutils
from wearables.scripts import data as weardata
from wearables.scripts import train as weartrain
from wearables.scripts import eval_ as weareval
from wearables.scripts import model as wearmodels
from wearables.scripts import DTW as weardtw

import torch
import torch.nn as nn
import torch.nn.functional as F

plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=1
plt.rcParams['savefig.dpi'] = 600
sns.set_style("ticks")

In [2]:
pfp = '/home/ngrav/project/wearables/results/'
savefigs = True

In [3]:
# load md info
from wearables.scripts.md_specification import mdpred_voi
from wearables.scripts.aesthetics import md_node_annotation

# filepaths to bst or pre-processed md with calculated metrics 
pp_md_fp = os.path.join(pfp, 'md_v522_220124.csv')

# load up to date md
md = pd.read_csv(pp_md_fp, index_col=0)

threshold = 10
md['Error group'] = 'lt{}wks'.format(threshold)
md.loc[(md['error'] >= threshold), 'Error group'] = 'Higher-than-actual'
md.loc[(md['error'] <= -threshold), 'Error group'] = 'Lower-than-actual'


In [7]:
mfp = '/home/ngrav/project/wearables/model_zoo'

# filepaths to bst or pre-processed md with calculated metrics 
bst_trainer = os.path.join(mfp, 'trainer_itv52_InceptionTime_GA5.pkl')
bst_modelpkl = os.path.join(mfp, '213-itv52_InceptionTime_GA5.pkl')
bst_modelembeds = os.path.join(pfp, 'embeds_v522_220124.csv')

# load up to date md
def loadpkl(fp):
    with open(fp, 'rb') as f:
        return pickle.load(f)
trainer = loadpkl(bst_trainer)

In [12]:
from scipy.stats import spearmanr

In [14]:
spearmanr(md['KPAS'], md['PQSI'])

SpearmanrResult(correlation=0.4050528286649845, pvalue=6.751823548751834e-98)

In [17]:
spearmanr(md.loc[md['ptb_37wks']==0, 'KPAS'], md.loc[md['ptb_37wks']==0., 'PQSI'])

SpearmanrResult(correlation=0.41048232526788914, pvalue=2.0456896208726e-89)

In [15]:
spearmanr(md.loc[md['ptb_37wks']==1, 'KPAS'], md.loc[md['ptb_37wks']==1, 'PQSI'])

SpearmanrResult(correlation=0.36198925595399467, pvalue=3.713947545994421e-10)

In [20]:
raw_md = pd.read_csv(os.path.join('/home/ngrav/data/wearables/raw/', 'MOD_Data_2021.csv'))

/home/ngrav/miniconda3/envs/wearables/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (16,22,36,77,97,117,137,195,215,235,255,275,315,335,426,440,462,463,465,466,485,506,536,577,581,625,626,627,668,671,673,674,697,700,701,703,704,705,708,709,712,713,730,798,799,804,807,808,812,817,818,822,880,881,885,886,890,891,908,1064,1067,1076,1084,1094,1095,1096,1097,1101,1102,1106,1132,1135,1136,1170,1250,1263,1577,1585,1586,1680,1739,1740,1741,1742,1743,1746,1747,1748,1749,1750,1870,1878,1879,1976,2166,2175,2188,2189,2203,2216,2217,2249,2250,2251,2252,2253,2256,2260,2261,2282,2283,2293,2294,2296,2297,2298,2299,2300,2303,2307,2308,2329,2340,2343,2344,2345,2346,2347,2350,2354,2355,2356,2376,2387,2390,2391,2392,2393,2394,2397,2401,2402,2423,2434,2437,2438,2439,2440,2441,2444,2448,2449,2450,2470,2471,2481,2484,2485,2486,2487,2488,2491,2495,2496,2517,2528,2531,2532,2533,2534,2535,2538,2542,2543,2544,2564,2575,2578,2579,2580,2581,2582,2585,2589,2590,2611,2622,2

In [27]:
dt = (pd.to_datetime(raw_md['address3_date']) - pd.to_datetime(raw_md['address1_date']))
dt.dropna(inplace=True)

In [36]:
dt.mean()

Timedelta('151 days 01:12:00')

In [35]:
151/30

5.033333333333333

In [21]:
raw_md.head()

,record_id,redcap_event_name,redcap_survey_identifier,customid,refused_future_research___1,refused_future_research___2,refused_future_research___3,reader,reader_text,age_enroll,...,edinb3_3trim,edinb4_3trim,edinb5_3trim,edinb6_3trim,edinb7_3trim,edinb8_3trim,edinb9_3trim,edinb10_3trim,edinburgh_check_trim3___1,third_edinburgh_depression_scale_complete
0,4,general_arm_1,NaN,4,0,0,0,7.0,Kristi,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,6,general_arm_1,NaN,6,0,0,0,4.0,NaN,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,9,general_arm_1,NaN,9,0,0,0,4.0,NaN,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,10,general_arm_1,NaN,10,0,0,0,4.0,NaN,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,21,general_arm_1,NaN,21,0,0,0,4.0,NaN,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [37]:
spearmanr(md.loc[md['ptb_37wks']==1, 'IS'], md.loc[md['ptb_37wks']==1, 'GA'])

SpearmanrResult(correlation=0.14690903752543452, pvalue=0.013531049839569587)

In [38]:
spearmanr(md.loc[md['ptb_37wks']==0, 'IS'], md.loc[md['ptb_37wks']==0, 'GA'])

SpearmanrResult(correlation=0.1951381140233743, pvalue=3.6980036316022404e-20)

In [6]:
md['gestage_by'].value_counts(dropna=False, normalize=True)

2.0    0.498985
1.0    0.288266
3.0    0.177020
0.0    0.019894
4.0    0.015834
Name: gestage_by, dtype: float64

In [10]:
trainer.data.data['cat_class_enc']

{'marital': {'0': 0.0, '1': 1.0},
 'gestage_by': {'-99.0': 0.0, '1.0': 1.0, '2.0': 2.0, '3.0': 3.0, '4.0': 4.0},
 'insur': {'-99.0': 0.0,
  '1.0': 1.0,
  '2.0': 2.0,
  '3.0': 3.0,
  '4.0': 4.0,
  '5.0': 5.0},
 'ethnicity': {'0': 0.0, '1': 1.0},
 'race': {'0': 0.0, '1': 1.0},
 'prior_ptb_all': {'0': 0.0, '1': 1.0},
 'fullterm_births': {'0': 0.0, '1': 1.0},
 'surghx_none': {'0.0': 0.0, '1.0': 1.0},
 'alcohol': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'smoke': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'drugs': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'hypertension': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'pregestational_diabetes': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'asthma_yes___1': {'0': 0.0, '1': 1.0},
 'asthma_yes___2': {'0': 0.0, '1': 1.0},
 'asthma_yes___3': {'0': 0.0, '1': 1.0},
 'asthma_yes___4': {'0': 0.0, '1': 1.0},
 'asthma_yes___5': {'0': 0.0, '1': 1.0},
 'asthma_yes___7': {'0': 0.0, '1': 1.0},
 'asthma_yes___8': {'0': 0.0, '1': 1.0},
 'asthma_yes___10': {'0': 0.0, '1': 1.0},
 'as